In [73]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import string
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [74]:
df = pd.read_csv('data/leg16-seances.csv')
data = pd.read_csv('data/leg16-seances.csv')

In [75]:
df

,Nom Orateur,ID Orateur,Texte,Thème Séance,ID Séance,Date Séance,ID Session,grammaire
0,Mme la présidente,719874,La séance est ouverte.,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,OUV_SEAN_2_1
1,Mme la présidente,719874,L’ordre du jour appelle la discussion du proje...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,ODJ_APPEL_DISCUSSION
2,Mme la présidente,719874,La parole est à Mme la ministre de la transiti...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,PAROLE_GENERIQUE
3,Mme Agnès Pannier-Runacher,759832,"Après la promulgation, vendredi, de la loi rel...",Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,PAROLE_GENERIQUE
4,M. Guillaume Kasbarian,719372,Elle a raison !,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,INTERRUPTION_1_10
...,...,...,...,...,...,...,...,...
312092,M. Gérard Leseul (SOC),774958,Je remercie le groupe GDR-NUPES d’avoir mis ce...,Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE
312093,Mme la présidente,720908,La parole est à M. le ministre.,Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE
312094,M. Stanislas Guerini,721498,Je le dis sous le contrôle de Mme la président...,Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE
312095,Mme la présidente,720908,Le débat est clos.,Réforme des retraites et pénibilité,RUANR5L16S2023IDS26837,lundi 27 février 2023,SCR5A2023O1,PAROLE_GENERIQUE


In [76]:
df['Nom Orateur'].value_counts(dropna = False)

Mme la présidente                                    81699
M. le président                                      10729
M. Éric Dupond-Moretti                                3333
M. Gabriel Attal                                      3002
M. Sylvain Maillard                                   2903
                                                     ...  
M. Christophe Bex et M. Laurent Alexandre                1
Mme Maud Bregeon et M. Sylvain Maillard                  1
Mme Elsa Faucillon (GDR)                                 1
M. Philippe Ballard                                      1
M. Joël Giraud et plusieurs députés du groupe Dem        1
Name: Nom Orateur, Length: 2255, dtype: int64

In [77]:
df = df[~df['Nom Orateur'].isin(['Mme la présidente', 'M. le président'])]

In [78]:
df['Nom Orateur'].value_counts(dropna = False)

M. Éric Dupond-Moretti                                                      3333
M. Gabriel Attal                                                            3002
M. Sylvain Maillard                                                         2903
M. Erwan Balanant                                                           2876
M. Benjamin Lucas                                                           2815
                                                                            ... 
M. Éric Bothorel, M. David Amiel, M. Sylvain Maillard et M. Bruno Studer       1
M. Fabien Di Filippo et Mme Ségolène Amiot                                     1
M. Dino Cinieri, M. Pierre Cordier et M. Maxime Minot                          1
M. Romain Daubié                                                               1
M. Éric Bothorel, rapporteur et M. Bruno Millienne                             1
Name: Nom Orateur, Length: 2253, dtype: int64

In [79]:
def drop_na(df, column):
    df = df.dropna(subset= [f'{column}'])
    return df

In [80]:
df = drop_na(data, 'Texte')

In [81]:
df.isnull().sum()

Nom Orateur     32
ID Orateur       0
Texte            0
Thème Séance     0
ID Séance        0
Date Séance      0
ID Session       0
grammaire        0
dtype: int64

In [82]:
df.shape

(311948, 8)

In [83]:
df.isnull().sum()

Nom Orateur     32
ID Orateur       0
Texte            0
Thème Séance     0
ID Séance        0
Date Séance      0
ID Session       0
grammaire        0
dtype: int64

In [92]:
def remove_short_sentences(df, n):
    def word_count(sentence):
        return len(sentence.split())
    df = df[df['Texte'].apply(word_count) >= n]
    return df

In [99]:
def apply_preprocessing(df, punct_opt = True):
    def preprocessing(sentence, punct_option = True):
        # Removing whitespaces
        sentence = sentence.strip()
        # Lowercasing
        sentence = sentence.lower()
        # Removing punctuation
        if punct_option == True:
            for punctuation in string.punctuation.replace('?','').replace('!',''):
                sentence = sentence.replace(punctuation, '')
        else:
            for punctuation in string.punctuation:
                sentence = sentence.replace(punctuation, '')
        return sentence
    df['Texte'] = df['Texte'].apply(preprocessing)
    df['Texte'] = df['Texte'].apply(text_to_word_sequence,filters=string.punctuation.replace('?','').replace('!',''), lower=True, split=' ')
    return df

In [100]:
df_test = df.head(2)
df_test

,Nom Orateur,ID Orateur,Texte,Thème Séance,ID Séance,Date Séance,ID Session,grammaire
0,Mme la présidente,719874,La séance est ouverte.,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,OUV_SEAN_2_1
1,Mme la présidente,719874,L’ordre du jour appelle la discussion du proje...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,ODJ_APPEL_DISCUSSION


In [101]:
apply_preprocessing(df_test)

/var/folders/mp/99nc3g2s3t192ft55tyrlzdh0000gn/T/ipykernel_56944/1105183339.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Texte'] = df['Texte'].apply(preprocessing)
/var/folders/mp/99nc3g2s3t192ft55tyrlzdh0000gn/T/ipykernel_56944/1105183339.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Texte'] = df['Texte'].apply(text_to_word_sequence,filters=string.punctuation.replace('?','').replace('!',''), lower=True, split=' ')


,Nom Orateur,ID Orateur,Texte,Thème Séance,ID Séance,Date Séance,ID Session,grammaire
0,Mme la présidente,719874,"[la, séance, est, ouverte]",Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,OUV_SEAN_2_1
1,Mme la présidente,719874,"[l’ordre, du, jour, appelle, la, discussion, d...",Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,ODJ_APPEL_DISCUSSION


In [102]:
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence


In [ ]:
test = text_to_word_sequence("I like cheese ?", filters=string.punctuation.replace('?','').replace('!',''), lower=True, split=' ')
test

['i', 'like', 'cheese', '?']